# 6_serve_model_with_Flask

In this notebook, we'll make a [Flask](https://www.fullstackpython.com/flask.html) web form application to show how our models can interact with a web application.

## Set up environment variables and load necessary libraries

In [ ]:
import os
PROJECT = "predict-babyweight-10142021"
BUCKET = PROJECT
REGION = "us-central1"

os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET 
os.environ["REGION"] = REGION

## Step 1: Deploy application

By default, the Cloud Shell doesn't have Flask installed, so copy over the following command to install it.

`python3 -m pip install --user Flask==0.12.1`

Next, we'll need to copy our web app to the Cloud Shell. We can use [Google Cloud Storage](https://cloud.google.com/storage) as an inbetween.

In [ ]:
%%bash
# Deploy a web application that consumes your model service on Cloud AI Platform
gsutil -m rm -r gs://$BUCKET/prediction_app
gsutil -m cp -r application/ gs://$BUCKET/prediction_app

Removing gs://predict-babyweight-10142021/prediction_app/.ipynb_checkpoints/Icon-checkpoint#1635001996151865...
Removing gs://predict-babyweight-10142021/prediction_app/.ipynb_checkpoints/app-checkpoint.yaml#1635001996168750...
Removing gs://predict-babyweight-10142021/prediction_app/.ipynb_checkpoints/main-checkpoint.py#1635001996061983...
Removing gs://predict-babyweight-10142021/prediction_app/Icon#1635001995963341...
Removing gs://predict-babyweight-10142021/prediction_app/main.py#1635001995982540...
Removing gs://predict-babyweight-10142021/prediction_app/app.yaml#1635001995948000...
Removing gs://predict-babyweight-10142021/prediction_app/static/.ipynb_checkpoints/baby-checkpoint.png#1635001996177767...
Removing gs://predict-babyweight-10142021/prediction_app/static/.ipynb_checkpoints/feet-checkpoint.png#1635001996130233...
Removing gs://predict-babyweight-10142021/prediction_app/static/feet.png#1635001995996596...
Removing gs://predict-babyweight-10142021/prediction_app/template

Run the below cell, and copy the output into the [Google Cloud Shell](https://console.cloud.google.com/home/dashboard?cloudshell=true)

In [ ]:
%%bash
echo rm -r prediction_app/
echo mkdir prediction_app/
echo gsutil cp -r gs://$BUCKET/prediction_app ./
echo python3 prediction_app/main.py

rm -r prediction_app/
mkdir prediction_app/
gsutil cp -r gs://predict-babyweight-10142021/prediction_app ./
python3 prediction_app/main.py


## Step 2: Use our website to generate predictions
Time to play with the website! The cloud shell should now say something like `* Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)`. Click on the `http` link to go to our shiny new website. Fill out the form and give it a minute or two to process its first prediction. After the first one, the rest of the predictions will be lightning fast.


## Cleaning up


In [ ]:
# Delete model version resource
! gcloud ai-platform versions delete $MODEL_VERSION --quiet --model $MODEL_NAME 

# Delete model resource
! gcloud ai-platform models delete $MODEL_NAME --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r $JOB_DIR

# If the training job is still running, cancel it
! gcloud ai-platform jobs cancel $JOB_NAME --quiet --verbosity critical

## Summary



In this lab, we have deployed a simple Flask web form application on App Engine that takes inputs, transforms them into features, and sends them to a model service on Cloud AI Platform to generate and return predictions.